In [1]:
!pip install xgboost

     |████████████████████████████████| 1.8 MB 2.7 MB/s eta 0:00:01


In [89]:
# Imports for the project
import pandas as pd
import numpy as np
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [90]:
 # Creating Variable names for pandas DataFrame
data = [] 
for i in range(2005,2016):
    data.append('deaths_'+str(i))

## resident_status

{'1': 'RESIDENTS', '2': 'INTRASTATE NONRESIDENTS', '3': 'INTERSTATE NONRESIDENTS', '4': 'FOREIGN RESIDENTS'}

## month_of_death

{'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May', '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November', '12': 'December'}

## current_data_year

{'2005': '2005'}

## race_recode_5

{'0': 'Other (Puerto Rico only)', '1': 'White', '2': 'Black', '3': 'American Indian', '4': 'Asian or Pacific Islander'}

## 358_cause_recode

In [91]:
keys = []
for i in range(2005,2006): 
    loc = r'/Users/rahulteja/Desktop/COURSE/sem 2/SAT-5165 Intro to Big Data/Project/archive/'+ str(i)+'_codes.json'   
    dic = ''
    with open(loc, 'r') as f:
        data = f.read()
        data = data.replace('NaN', 'np.nan')
        dic=data
    d = eval(dic)
    keys.append(d)

In [92]:
def keyFinder(dic,lst):
    for key in dic:
        dic[key] = str(dic[key]).lower()
    keys_lst = []
    for key, sent in dic.items():
        for word in lst:
            if word in sent:
                keys_lst.append(int(key))
    return keys_lst

In [93]:
# Heart disese list
heart_disese_keys = keyFinder(keys[0]['358_cause_recode'],["heart"])
cancer_disese_keys = keyFinder(keys[0]['358_cause_recode'],["malignant","benign","Leukemia"])
respiratory_disese_keys = keyFinder(keys[0]['358_cause_recode'],["chronic lower","respiratory"])
stroke_disese_keys = keyFinder(keys[0]['358_cause_recode'],["stroke","cerebrovascular"])
accidents_disese_keys = keyFinder(keys[0]['358_cause_recode'],["accidents"])
diabetes_disese_keys = keyFinder(keys[0]['358_cause_recode'],["diabetes"])
oldAge_disese_keys = keyFinder(keys[0]['358_cause_recode'],["alzheimer=s","parkinson=s"])

In [94]:
diseases = {'heart':heart_disese_keys,'cancer':cancer_disese_keys,'respiratory':respiratory_disese_keys,'stroke':stroke_disese_keys,'accidents':accidents_disese_keys,'diabetes':diabetes_disese_keys,'oldAge':oldAge_disese_keys}
diseases_keys= set(heart_disese_keys+cancer_disese_keys+respiratory_disese_keys+stroke_disese_keys+accidents_disese_keys+diabetes_disese_keys+oldAge_disese_keys)

In [95]:
diseases

{'heart': [94,
  198,
  200,
  204,
  207,
  209,
  210,
  212,
  213,
  215,
  216,
  217,
  219,
  220,
  229,
  230,
  231,
  233,
  369],
 'cancer': [50,
  70,
  71,
  76,
  90,
  96,
  97,
  98,
  99,
  100,
  104,
  105,
  112,
  116,
  120,
  121,
  124,
  125,
  126,
  131,
  132,
  141],
 'respiratory': [13,
  90,
  95,
  247,
  248,
  251,
  252,
  258,
  260,
  261,
  262,
  262,
  276,
  278,
  361,
  362,
  371,
  410],
 'stroke': [234, 238, 239],
 'accidents': [382, 383, 384, 385, 397, 398, 399, 400, 401, 402, 423],
 'diabetes': [159],
 'oldAge': [188, 189]}

In [96]:
#list of dataframes from 2005 to 2015
Overall_data = []
for i,j in zip(range(2005,2016),data): #change range from 2005 to 2016
    loc = r'/Users/rahulteja/Desktop/COURSE/sem 2/SAT-5165 Intro to Big Data/Project/archive/'+str(i)+'_data.csv'
    j = pd.read_csv(loc)[['current_data_year','month_of_death','resident_status','sex','race_recode_5','358_cause_recode']]
    j.dropna(how='any',axis=1,inplace=True) #Droping Null Values
    j = j[j['358_cause_recode'].isin(diseases_keys)]
    Overall_data.append(j)
    

/Users/rahulteja/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (39,40,41,42,43,44,45,46,47,59,60,61,62,63,64,65,66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/rahulteja/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (39,40,41,42,43,44,45,46,47,59,60,61,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/rahulteja/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (39,40,41,42,43,44,45,46,59,60,61,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/rahulteja/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: Dtype

In [97]:
data = pd.concat(Overall_data).reset_index()

In [98]:
data.shape

(8593283, 7)

In [99]:
data.head()

,index,current_data_year,month_of_death,resident_status,sex,race_recode_5,358_cause_recode
0,0,2005,1,1,F,1,98
1,2,2005,1,1,F,1,239
2,3,2005,1,1,M,1,159
3,5,2005,1,1,F,3,239
4,9,2005,1,3,F,1,125


# Feature Engineering

In [100]:
data_1 = data[['resident_status','sex','race_recode_5','358_cause_recode']]
data_1['month'] = (data['current_data_year']-2005)*12+data['month_of_death']

/var/folders/wm/wtzrkjwd6y99x1nbq6srlzkh0000gn/T/ipykernel_76910/3239279132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1['month'] = (data['current_data_year']-2005)*12+data['month_of_death']


In [101]:
data_1.head()

,resident_status,sex,race_recode_5,358_cause_recode,month
0,1,F,1,98,1
1,1,F,1,239,1
2,1,M,1,159,1
3,1,F,3,239,1
4,3,F,1,125,1


In [102]:
data_1['sex'] = data_1['sex'].replace({'M': 1, 'F': 0})

/var/folders/wm/wtzrkjwd6y99x1nbq6srlzkh0000gn/T/ipykernel_76910/1988617779.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1['sex'] = data_1['sex'].replace({'M': 1, 'F': 0})


In [103]:
data_1['count']=1

/var/folders/wm/wtzrkjwd6y99x1nbq6srlzkh0000gn/T/ipykernel_76910/2922386101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1['count']=1


In [104]:
data_1

,resident_status,sex,race_recode_5,358_cause_recode,month,count
0,1,0,1,98,1,1
1,1,0,1,239,1,1
2,1,1,1,159,1,1
3,1,0,3,239,1,1
4,3,0,1,125,1,1
...,...,...,...,...,...,...
8593278,1,1,1,215,132,1
8593279,1,1,2,238,132,1
8593280,1,0,2,209,131,1
8593281,1,0,1,215,132,1


In [105]:
diseases

{'heart': [94,
  198,
  200,
  204,
  207,
  209,
  210,
  212,
  213,
  215,
  216,
  217,
  219,
  220,
  229,
  230,
  231,
  233,
  369],
 'cancer': [50,
  70,
  71,
  76,
  90,
  96,
  97,
  98,
  99,
  100,
  104,
  105,
  112,
  116,
  120,
  121,
  124,
  125,
  126,
  131,
  132,
  141],
 'respiratory': [13,
  90,
  95,
  247,
  248,
  251,
  252,
  258,
  260,
  261,
  262,
  262,
  276,
  278,
  361,
  362,
  371,
  410],
 'stroke': [234, 238, 239],
 'accidents': [382, 383, 384, 385, 397, 398, 399, 400, 401, 402, 423],
 'diabetes': [159],
 'oldAge': [188, 189]}

In [106]:
cause = {}
cancer = [50,70,71,76,90,96,97,98,99,100,104,105,112,116,120,121,124,125,126,131,132,141]
heart = [94,198,200,204,207,209,210,212,213,215,216,217,219,220,229,230,231,233,369]
respiratory=[13,90,95,247,248,251,252,258,260,261,262,262,276,278,361,362,371,410]
stroke=[234, 238, 239]
accidents=[382, 383, 384, 385, 397, 398, 399, 400, 401, 402, 423]
diabetes=[159]
oldAge=[188, 189]

for i in range(1,500):
    if i in cancer:
        cause[i] = 'cancer'
    elif i in heart:
        cause[i] = 'heart'
    elif i in respiratory:
        cause[i] = 'respiratory'
    elif i in stroke:
        cause[i] = 'stroke'
    elif i in accidents:
        cause[i] = 'accidents'
    elif i in diabetes:
        cause[i] = 'diabetes'
    elif i in oldAge:
        cause[i] = 'oldAge'


In [107]:
cause

{13: 'respiratory',
 50: 'cancer',
 70: 'cancer',
 71: 'cancer',
 76: 'cancer',
 90: 'cancer',
 94: 'heart',
 95: 'respiratory',
 96: 'cancer',
 97: 'cancer',
 98: 'cancer',
 99: 'cancer',
 100: 'cancer',
 104: 'cancer',
 105: 'cancer',
 112: 'cancer',
 116: 'cancer',
 120: 'cancer',
 121: 'cancer',
 124: 'cancer',
 125: 'cancer',
 126: 'cancer',
 131: 'cancer',
 132: 'cancer',
 141: 'cancer',
 159: 'diabetes',
 188: 'oldAge',
 189: 'oldAge',
 198: 'heart',
 200: 'heart',
 204: 'heart',
 207: 'heart',
 209: 'heart',
 210: 'heart',
 212: 'heart',
 213: 'heart',
 215: 'heart',
 216: 'heart',
 217: 'heart',
 219: 'heart',
 220: 'heart',
 229: 'heart',
 230: 'heart',
 231: 'heart',
 233: 'heart',
 234: 'stroke',
 238: 'stroke',
 239: 'stroke',
 247: 'respiratory',
 248: 'respiratory',
 251: 'respiratory',
 252: 'respiratory',
 258: 'respiratory',
 260: 'respiratory',
 261: 'respiratory',
 262: 'respiratory',
 276: 'respiratory',
 278: 'respiratory',
 361: 'respiratory',
 362: 'respiratory'

In [108]:
len(cause)

74

In [109]:
data3=data_1[['resident_status','sex','race_recode_5','358_cause_recode','month']]

In [110]:
data3

,resident_status,sex,race_recode_5,358_cause_recode,month
0,1,0,1,98,1
1,1,0,1,239,1
2,1,1,1,159,1
3,1,0,3,239,1
4,3,0,1,125,1
...,...,...,...,...,...
8593278,1,1,1,215,132
8593279,1,1,2,238,132
8593280,1,0,2,209,131
8593281,1,0,1,215,132


In [111]:
data3['Cause'] = data3['358_cause_recode'].apply(lambda x: cause[x])

In [112]:
data3

,resident_status,sex,race_recode_5,358_cause_recode,month,Cause
0,1,0,1,98,1,cancer
1,1,0,1,239,1,stroke
2,1,1,1,159,1,diabetes
3,1,0,3,239,1,stroke
4,3,0,1,125,1,cancer
...,...,...,...,...,...,...
8593278,1,1,1,215,132,heart
8593279,1,1,2,238,132,stroke
8593280,1,0,2,209,131,heart
8593281,1,0,1,215,132,heart


In [113]:
data3['Cause'] = data3['Cause'].replace({'cancer': 1, 'heart': 2, 'respiratory': 3,'stroke': 4, 'accidents': 5,'diabetes': 6, 'oldAge': 7})


In [114]:
data3

,resident_status,sex,race_recode_5,358_cause_recode,month,Cause
0,1,0,1,98,1,1
1,1,0,1,239,1,4
2,1,1,1,159,1,6
3,1,0,3,239,1,4
4,3,0,1,125,1,1
...,...,...,...,...,...,...
8593278,1,1,1,215,132,2
8593279,1,1,2,238,132,4
8593280,1,0,2,209,131,2
8593281,1,0,1,215,132,2


In [116]:
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')
sns.kdeplot(np.array(data3), index=[0], bw=0.5)

ValueError: If using all scalar values, you must pass an index

In [ ]:
data3['count']=1

In [ ]:
data3

In [ ]:
data_3 = data3[['month','resident_status','sex','race_recode_5','Cause','count']].groupby(['month','resident_status','sex','race_recode_5','Cause']).agg('sum').reset_index()
                                                                                                                                                                                                

In [ ]:
data_3

In [ ]:
def avgcal(df, groupby_cols, agg_col, new_col_name):
    df_copy = df.copy()
    groupby_cols.append(agg_col)
    avg_df = df_copy[groupby_cols].groupby(groupby_cols[:-1]).agg('mean').reset_index()
    avg_df.columns = groupby_cols[:-1] + [new_col_name]
    df_copy = pd.merge(df_copy, avg_df, on=groupby_cols[:-1], how='left')

    return df_copy


In [ ]:
data_3=avgcal(data_3, ['month','resident_status'], 'count', 'month_resstatus_avg_death')

In [ ]:
data_3=avgcal(data_3, ['month','sex'], 'count', 'month_sex_avg_death')

In [ ]:
data_3=avgcnt=avgcal(data_3, ['month','race_recode_5'], 'count', 'month_race_avg_death')

In [ ]:
data_3=avgcal(data_3, ['month','Cause'], 'count', 'month_cause_avg_death')

In [44]:
data_3

,month,resident_status,sex,race_recode_5,Cause,count,month_resstatus_avg_death,month_sex_avg_death,month_race_avg_death,month_cause_avg_death
0,1,1,0,1,1,4430,1104.982143,443.904255,1188.333333,350.777778
1,1,1,0,1,2,13009,1104.982143,443.904255,1188.333333,1100.633333
2,1,1,0,1,3,1137,1104.982143,443.904255,1188.333333,120.259259
3,1,1,0,1,4,4575,1104.982143,443.904255,1188.333333,394.960000
4,1,1,0,1,5,495,1104.982143,443.904255,1188.333333,89.851852
...,...,...,...,...,...,...,...,...,...,...
24183,132,4,1,2,6,1,3.933333,350.570000,170.294118,254.518519
24184,132,4,1,4,1,2,3.933333,350.570000,42.319149,330.620690
24185,132,4,1,4,2,6,3.933333,350.570000,42.319149,998.200000
24186,132,4,1,4,3,1,3.933333,350.570000,42.319149,138.607143


In [45]:
#Creating the lag features
def create_lag_feature(df,col,lags):
    tmp = df[['month','resident_status','sex','race_recode_5','Cause',col]]
    for lag in lags:
        shifted = tmp.copy()
        shifted['month'] = shifted['month'].apply(lambda x: x+lag)
        shifted.columns = ['month','resident_status','sex','race_recode_5','Cause',col+'_lag_'+str(lag)]
        df = pd.merge(df,shifted,on=['month','resident_status','sex','race_recode_5','Cause'],how='left')
    return df

In [46]:
data_3 = create_lag_feature(data_3,'count',[1,3,6,9,12])

In [47]:
for column in ['month_resstatus_avg_death','month_sex_avg_death','month_race_avg_death','month_cause_avg_death']:
    data_3 = create_lag_feature(data_3,column,[1,3,6,9,12])

In [48]:
data_3

,month,resident_status,sex,race_recode_5,Cause,count,month_resstatus_avg_death,month_sex_avg_death,month_race_avg_death,month_cause_avg_death,...,month_race_avg_death_lag_1,month_race_avg_death_lag_3,month_race_avg_death_lag_6,month_race_avg_death_lag_9,month_race_avg_death_lag_12,month_cause_avg_death_lag_1,month_cause_avg_death_lag_3,month_cause_avg_death_lag_6,month_cause_avg_death_lag_9,month_cause_avg_death_lag_12
0,1,1,0,1,1,4430,1104.982143,443.904255,1188.333333,350.777778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0,1,2,13009,1104.982143,443.904255,1188.333333,1100.633333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,0,1,3,1137,1104.982143,443.904255,1188.333333,120.259259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,0,1,4,4575,1104.982143,443.904255,1188.333333,394.960000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,0,1,5,495,1104.982143,443.904255,1188.333333,89.851852,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24183,132,4,1,2,6,1,3.933333,350.570000,170.294118,254.518519,...,NaN,NaN,NaN,187.104167,181.448980,NaN,NaN,NaN,263.296296,265.629630
24184,132,4,1,4,1,2,3.933333,350.570000,42.319149,330.620690,...,NaN,37.818182,NaN,NaN,36.163265,NaN,361.000000,NaN,NaN,366.074074
24185,132,4,1,4,2,6,3.933333,350.570000,42.319149,998.200000,...,36.795918,37.818182,35.0,40.863636,36.163265,928.066667,881.433333,928.758621,1059.833333,1086.068966
24186,132,4,1,4,3,1,3.933333,350.570000,42.319149,138.607143,...,NaN,NaN,35.0,NaN,NaN,NaN,NaN,126.407407,NaN,NaN


In [49]:
data_3.fillna(0,inplace = True)
data_3

,month,resident_status,sex,race_recode_5,Cause,count,month_resstatus_avg_death,month_sex_avg_death,month_race_avg_death,month_cause_avg_death,...,month_race_avg_death_lag_1,month_race_avg_death_lag_3,month_race_avg_death_lag_6,month_race_avg_death_lag_9,month_race_avg_death_lag_12,month_cause_avg_death_lag_1,month_cause_avg_death_lag_3,month_cause_avg_death_lag_6,month_cause_avg_death_lag_9,month_cause_avg_death_lag_12
0,1,1,0,1,1,4430,1104.982143,443.904255,1188.333333,350.777778,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,0,1,2,13009,1104.982143,443.904255,1188.333333,1100.633333,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,1,0,1,3,1137,1104.982143,443.904255,1188.333333,120.259259,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,1,0,1,4,4575,1104.982143,443.904255,1188.333333,394.960000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,1,0,1,5,495,1104.982143,443.904255,1188.333333,89.851852,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24183,132,4,1,2,6,1,3.933333,350.570000,170.294118,254.518519,...,0.000000,0.000000,0.0,187.104167,181.448980,0.000000,0.000000,0.000000,263.296296,265.629630
24184,132,4,1,4,1,2,3.933333,350.570000,42.319149,330.620690,...,0.000000,37.818182,0.0,0.000000,36.163265,0.000000,361.000000,0.000000,0.000000,366.074074
24185,132,4,1,4,2,6,3.933333,350.570000,42.319149,998.200000,...,36.795918,37.818182,35.0,40.863636,36.163265,928.066667,881.433333,928.758621,1059.833333,1086.068966
24186,132,4,1,4,3,1,3.933333,350.570000,42.319149,138.607143,...,0.000000,0.000000,35.0,0.000000,0.000000,0.000000,0.000000,126.407407,0.000000,0.000000


In [50]:
onehot_race = pd.get_dummies(data_3.race_recode_5, prefix='race')
data_3 = pd.concat([data_3, onehot_race],axis=1)

In [51]:
onehot_cause = pd.get_dummies(data_3.Cause, prefix='cause')
data_3 = pd.concat([data_3, onehot_cause],axis=1)

In [52]:
onehot_res = pd.get_dummies(data_3.resident_status, prefix='res_status')
data_3 = pd.concat([data_3, onehot_res],axis=1)

In [53]:
data_3

,month,resident_status,sex,race_recode_5,Cause,count,month_resstatus_avg_death,month_sex_avg_death,month_race_avg_death,month_cause_avg_death,...,cause_2,cause_3,cause_4,cause_5,cause_6,cause_7,res_status_1,res_status_2,res_status_3,res_status_4
0,1,1,0,1,1,4430,1104.982143,443.904255,1188.333333,350.777778,...,0,0,0,0,0,0,1,0,0,0
1,1,1,0,1,2,13009,1104.982143,443.904255,1188.333333,1100.633333,...,1,0,0,0,0,0,1,0,0,0
2,1,1,0,1,3,1137,1104.982143,443.904255,1188.333333,120.259259,...,0,1,0,0,0,0,1,0,0,0
3,1,1,0,1,4,4575,1104.982143,443.904255,1188.333333,394.960000,...,0,0,1,0,0,0,1,0,0,0
4,1,1,0,1,5,495,1104.982143,443.904255,1188.333333,89.851852,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24183,132,4,1,2,6,1,3.933333,350.570000,170.294118,254.518519,...,0,0,0,0,1,0,0,0,0,1
24184,132,4,1,4,1,2,3.933333,350.570000,42.319149,330.620690,...,0,0,0,0,0,0,0,0,0,1
24185,132,4,1,4,2,6,3.933333,350.570000,42.319149,998.200000,...,1,0,0,0,0,0,0,0,0,1
24186,132,4,1,4,3,1,3.933333,350.570000,42.319149,138.607143,...,0,1,0,0,0,0,0,0,0,1


In [54]:
data_3.columns

Index(['month', 'resident_status', 'sex', 'race_recode_5', 'Cause', 'count',
       'month_resstatus_avg_death', 'month_sex_avg_death',
       'month_race_avg_death', 'month_cause_avg_death', 'count_lag_1',
       'count_lag_3', 'count_lag_6', 'count_lag_9', 'count_lag_12',
       'month_resstatus_avg_death_lag_1', 'month_resstatus_avg_death_lag_3',
       'month_resstatus_avg_death_lag_6', 'month_resstatus_avg_death_lag_9',
       'month_resstatus_avg_death_lag_12', 'month_sex_avg_death_lag_1',
       'month_sex_avg_death_lag_3', 'month_sex_avg_death_lag_6',
       'month_sex_avg_death_lag_9', 'month_sex_avg_death_lag_12',
       'month_race_avg_death_lag_1', 'month_race_avg_death_lag_3',
       'month_race_avg_death_lag_6', 'month_race_avg_death_lag_9',
       'month_race_avg_death_lag_12', 'month_cause_avg_death_lag_1',
       'month_cause_avg_death_lag_3', 'month_cause_avg_death_lag_6',
       'month_cause_avg_death_lag_9', 'month_cause_avg_death_lag_12', 'race_1',
       'race_2',

# Model building

In [55]:
train_col=['month', 'sex', 'count_lag_1',
       'count_lag_3', 'count_lag_6', 'count_lag_9', 'count_lag_12',
       'month_resstatus_avg_death_lag_1', 'month_resstatus_avg_death_lag_3',
       'month_resstatus_avg_death_lag_6', 'month_resstatus_avg_death_lag_9',
       'month_resstatus_avg_death_lag_12', 'month_sex_avg_death_lag_1',
       'month_sex_avg_death_lag_3', 'month_sex_avg_death_lag_6',
       'month_sex_avg_death_lag_9', 'month_sex_avg_death_lag_12',
       'month_race_avg_death_lag_1', 'month_race_avg_death_lag_3',
       'month_race_avg_death_lag_6', 'month_race_avg_death_lag_9',
       'month_race_avg_death_lag_12', 'month_cause_avg_death_lag_1',
       'month_cause_avg_death_lag_3', 'month_cause_avg_death_lag_6',
       'month_cause_avg_death_lag_9', 'month_cause_avg_death_lag_12', 'race_1',
       'race_2', 'race_3', 'race_4', 'cause_1', 'cause_2', 'cause_3',
       'cause_4', 'cause_5', 'cause_6', 'cause_7', 'res_status_1',
       'res_status_2', 'res_status_3', 'res_status_4']

In [56]:
train = data_3[data_3.month<132]
train_X = train[train_col]
train_y = train['count']
test = data_3[data_3.month==132]
test_X = test[train_col]
test_y = test['count']

In [57]:
train_X.columns

Index(['month', 'sex', 'count_lag_1', 'count_lag_3', 'count_lag_6',
       'count_lag_9', 'count_lag_12', 'month_resstatus_avg_death_lag_1',
       'month_resstatus_avg_death_lag_3', 'month_resstatus_avg_death_lag_6',
       'month_resstatus_avg_death_lag_9', 'month_resstatus_avg_death_lag_12',
       'month_sex_avg_death_lag_1', 'month_sex_avg_death_lag_3',
       'month_sex_avg_death_lag_6', 'month_sex_avg_death_lag_9',
       'month_sex_avg_death_lag_12', 'month_race_avg_death_lag_1',
       'month_race_avg_death_lag_3', 'month_race_avg_death_lag_6',
       'month_race_avg_death_lag_9', 'month_race_avg_death_lag_12',
       'month_cause_avg_death_lag_1', 'month_cause_avg_death_lag_3',
       'month_cause_avg_death_lag_6', 'month_cause_avg_death_lag_9',
       'month_cause_avg_death_lag_12', 'race_1', 'race_2', 'race_3', 'race_4',
       'cause_1', 'cause_2', 'cause_3', 'cause_4', 'cause_5', 'cause_6',
       'cause_7', 'res_status_1', 'res_status_2', 'res_status_3',
       'res_stat

In [58]:
cv_params = {'n_estimators':[300,400,500], 'learning_rate':np.arange(0.3,0.51,0.01)}
xgb_model = xgb.XGBRegressor(max_depth=8, subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.8,seed=42)
cv1 = GridSearchCV(estimator=xgb_model, param_grid=cv_params, scoring='neg_root_mean_squared_error', cv=5, verbose=1)
cv1.fit(train_X, train_y)

Fitting 5 folds for each of 63 candidates, totalling 315 fits


KeyboardInterrupt: 

In [54]:
cv1.best_params_, cv1.best_score_

({'learning_rate': 0.31, 'n_estimators': 500}, -182.19975751377427)

# FINAL MODEL

In [55]:
cv_xgb_model = xgb.XGBRegressor(n_estimators=500,learning_rate=0.5,max_depth=8,
                            subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.8,colsample_bylevel=1,
                            gamma=100, min_child_weight=1, reg_lambda=1, seed=42)
cv_xgb_model.fit(train_X,train_y, eval_set=[(train_X,train_y),(test_X,test_y)],eval_metric='rmse',early_stopping_rounds=10)

[0]	validation_0-rmse:649.28468	validation_1-rmse:744.79691
[1]	validation_0-rmse:344.43134	validation_1-rmse:404.46634
[2]	validation_0-rmse:191.40168	validation_1-rmse:245.41672
[3]	validation_0-rmse:120.47030	validation_1-rmse:158.66182


C:\Users\dhanu\Anaconda3\envs\cs4821\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\dhanu\Anaconda3\envs\cs4821\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[4]	validation_0-rmse:80.14586	validation_1-rmse:143.31828
[5]	validation_0-rmse:63.04121	validation_1-rmse:127.10498
[6]	validation_0-rmse:52.07575	validation_1-rmse:108.17474
[7]	validation_0-rmse:44.18282	validation_1-rmse:101.74294
[8]	validation_0-rmse:41.42280	validation_1-rmse:96.86126
[9]	validation_0-rmse:39.76476	validation_1-rmse:96.26561
[10]	validation_0-rmse:38.19548	validation_1-rmse:91.93105
[11]	validation_0-rmse:36.98991	validation_1-rmse:81.70655
[12]	validation_0-rmse:36.64980	validation_1-rmse:81.53993
[13]	validation_0-rmse:36.28669	validation_1-rmse:81.51012
[14]	validation_0-rmse:35.99499	validation_1-rmse:83.32626
[15]	validation_0-rmse:35.43519	validation_1-rmse:81.33423
[16]	validation_0-rmse:34.93990	validation_1-rmse:80.59857
[17]	validation_0-rmse:32.33193	validation_1-rmse:80.53421
[18]	validation_0-rmse:31.96288	validation_1-rmse:80.82013
[19]	validation_0-rmse:27.11103	validation_1-rmse:80.88864
[20]	validation_0-rmse:23.73173	validation_1-rmse:80.80609

XGBRegressor(base_score=None, booster=None, callbacks=None, colsample_bylevel=1,
             colsample_bynode=0.8, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=100, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.5, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)